---
title: "Presentation: Nautilopy applied to the 3D mapping of underwater caves"
subtitle: Overview of the work realized with Nautilopy on the underwater cave dataset
authors:
  - name: Thomas Guilment
    affiliations:
      - University of Louisiana at Lafayette
    email: thomas.guilment@gmail.com
    corresponding: true
    orcid: 0009-0003-8163-3976
  - name: Gabriele Morra
    affiliations:
      - University of Louisiana at Lafayette
  - name: Leonardo Macelloni
    affiliations:
      - University of Mississippi
  - name: Marco D'Emilio
    affiliations:
      - University of Southern University
format:
  html:
    code-fold: true
---

![](./img/Logo_nautilopy_tiny.png)

(theUnderwaterCaveDataset)=
## The underwater cave dataset
NautiloPy already includes a `csv.zip` file in the data folder, which is used for the entire project.  
The original data are available at [https://cirs.udg.edu/caves-dataset/](https://cirs.udg.edu/caves-dataset/). To access the corresponding article, see also [Maillos et al., 2017](doi:10.1177/0278364917732838).

### Abstract from the authors of the dataset

From [Maillos et al., 2017](doi:10.1177/0278364917732838)

"We provide a data set collected with an autonomous underwater vehicle (AUV) tested in the unstructured environment of an underwater cave complex. The vehicle is equipped with two mechanically scanned imaging sonar (MSIS) to simultaneously map the caves' horizontal and vertical surfaces, a Doppler velocity log (DvL), two inertial measurement units (IMUs), a depth sensor, and a vertically mounted camera imaging the sea-floor for ground truth validation in specific points. Here we present data using this testbed that was collected while guided by a diver, due to the caves' spatial complexity, during July 2013. For ease of use, the original ROS bag files are accompanied by a derivative version combining imagery and human-readable text files for processing on other environments."

:::{figure}
:label: my-figure
:align: left

(aerial-view)=
![The approximate path traveled underwater overlaid on an aerial image from Google Earth.](./img/CavesGoogleEarth2.jpg)

(auv-config)=
![AUV configuration](./img/auv_config.png)

Aerial image showing the AUV trajectory and AUV configuration from [Maillos et al., 2017](doi:10.1177/0278364917732838).
:::

::: {.callout-warning}
#### Correction in Super Seaking Reference axis
We believe that there is an error in the 3D axis representation of the vertical SONAR. The X-axis (in red) should point to the seafloor (not the sky). In this work, a 180° degrees rotation around the Z axis was applied.
:::

:::{note} Dataset in details
:class: dropdown
%:header: Available data from sensors in CSV file
- **/depth_sensor:** DS2806 HPS-A pressure sensor data.
- **/dvl_linkquest:** LinkQuest NavQuest 600 sensor data. Contains bottom and water-referenced velocities and other important sensor data.
- **/imu_adis:** Analog Devices ADIS16480 sensor data. Contains orientation as both Euler angles and quaternion. It also contains raw data for accelerometers, gyros, magnetometers, and temperature, and an estimation of the gyro biases.
- **/imu_adis_ros:** Analog Devices ADIS16480 sensor orientation [using standard IMU ROS message.](http://wiki.ros.org/sensor_msgs)
- **/imu_xsens_mti:** Xsens MTi sensor data. Same message type and the same information as the topic /imu_adis.
- **/imu_xsens_mti_ros:** Xsens MTi sensor orientation using standard IMU ROS message.
- **/odometry:** estimation of the robot pose provided as a [standard Odometry ROS message.](http://wiki.ros.org/nav_msgs)
- **/sonar_micron:** Tritech Micron DST sensor beam data.
- **/sonar_micron_ros:** Micron data provided as [standard Laserscan ROS message.](http://wiki.ros.org/sensor_msgs)
- **/sonar_seaking:** Tritech Super SeaKing DFP profiler sensor beam data.
- **/sonar_seaking_ros:** profiler data provided as standard Laserscan ROS message.
- **/tf:** [standard ROS messages](http://wiki.ros.org/tf) containing sensor offsets.

```{figure} ./img/sensors_offset.png
:name: Sensors_offset
:align: left
Sensors offsets
```
:::

### Import Nautilopy

In [ ]:
from nautilopy.core import *

(prepareTheData)=
## Prepare the data
For completeness, instead of using the human format text file given by the dataset authors, the ROS bag data were extracted using the Python package `Bagpy` (see this [notebook](#ROS2csv) to see how it was done), and each topic was converted into CSV files. 

In this project, the odometry field is considered to be the best estimate of pose and orientation.
Therefore, the pre-processing is about interpolating the odometry to fit the vertical and horizontal sonar data (same timestamps).

Each processed data is saved as a pickle file.
For more information, don't hesitate to check the `01_Preprocessing.ipynb` notebook by clicking the [link below](#01_Preprocessing). 

:::{warning} Correction
:icon:False
During the data pre-processing phase, the vertical sonar orientation 3D axis has proven to be wrong, which is why there is a correction at the end of this external notebook.
:::

### Unzip the CSV dataset folder

In [ ]:
# If the data are not unzip
if not os.path.isdir("./data/csv/"):
    # Unizp the csv.zip file containing the CSV files
    f_unzip_file(s_zip_path = os.path.join("data", "csv.zip"), s_extract_path = os.path.join("data", '.'))
else:
    print("""The csv folder already exist.""")

### Create the Pickle dataset from the CSVs

In [ ]:
# Test if the pickle_dataset already exist
if not os.path.isdir("./data/Pickle_dataset/"):
    %run 01_Preprocessing.ipynb
else:
    print("""The folder Pickle_dataset already exists. 
If you want to recreate the Pickle dataset, delete the Pickle_dataset in the data folder, then re-run this cell.""")

### Load the pre-processed data
A little trick is used to get the `globals()` (local variables in the workspace) from an outside function by passing the local `globals` variables from the current notebook. The pickle variables are loaded in the current notebook. This is like loading a common Matlab `workspace`.

In [ ]:
# List the pickle files
l_var = os.listdir(os.path.join('.','data','Pickle_dataset'))

# Load only the needed variables
glob = globals()
f_load_var(os.path.join('.','data','Pickle_dataset'), glob, 
           ['m_xyz_pos', # XYZ AUV position
            'm_ypr', # Yaw, Pitch, Roll associated with the AUV positions
            'v_timestamp.pkl']) # Timestamps associated with AUV positions

# Deleting glob
del glob

(VisualizationOfTheAuvOrientationOverTheTrajectory)=
## Visualization of the AUV orientation over the trajectory
This part presents different solutions to visualize the AUV orientation over its trajectory.  
We are using the variable `m_xyz_pos` {eval}`m_xyz_pos.shape` and `m_ypr` {eval}`m_ypr.shape` created from the `odometry` data to display the trajectory and orientation of the AUV.

### Traditional multi-view

In [ ]:
# Inline mode that displays static images inside the notebook
%matplotlib inline
d_start = 9000      # Starting point (from 0 to np.shape(m_xyz_pos)[0])
d_end = 11000       # Ending point (maximum value is np.shape(m_xyz_pos)[0])
d_downsampling = 20 # For faster display, only shows 1 point every 20 points
d_step = 300        # (default 200) Show the pose every 300 divided by downsampled points
d_scale = 1.5       # (default 1.5) Length of the AUV 3D axis 
d_max_ticks = 8     # (default 5) number of ticks per axis

f_plot_auv_orientation_subplots(
    m_xyz_pos[d_start:d_end:d_downsampling], # XYZ AUV position
    m_ypr[d_start:d_end:d_downsampling], # Yaw, Pitch, Roll associated with the AUV positions
    d_step=int(d_step/d_downsampling), # updated step depending of downsampling
    d_scale = d_scale, 
    d_max_ticks = d_max_ticks
)

## Interactive AUV orientation over trajectory
This section presents two interactive plots of the Autonomous 3D orientation axis over its trajectory.
First, we use `Matplotlib`, then `plotly`.

In [ ]:
d_start = 0
d_end = np.shape(m_xyz_pos)[0]
d_downsampling = 10 # For faster and lighter display, only shows 1 point every 10 points
d_step = 400 # (default 200) Show the pose every 400 divided by downsampled points
d_scale = 3 # (default 1.5) Length of the AUV 3D axis 
d_max_ticks = 8 # (default 5) number of ticks per axis

v_figsize =(12, 5) # (default (12,5)) Width size and height size of the main figure

o_anim_auv_orientation = f_create_auv_orientation_animation(
    m_xyz_pos[d_start:d_end:d_downsampling],
    m_ypr[d_start:d_end:d_downsampling],
    d_step=int(d_step/d_downsampling),
    d_scale = d_scale,
    d_max_ticks = d_max_ticks,
    v_figsize = v_figsize
)

In [ ]:
# Convert the animation into javascript then HTML in one line code
display(HTML(o_anim_auv_orientation.to_jshtml()))

In [ ]:
# Ensure to be back to the inline backend
%matplotlib inline

# Clean widgets in memory
plt.close('all')

In [ ]:
# Use the renderers "plotly_mimetype", "Jupyterlab", and "notebook" to ensure compatibility for the display animations
# available renderers at https://plot.ly/python/renderers/
import plotly.io as pio

# Use Jupyterlab compatible renderers
pio.renderers.default = "jupyterlab+notebook+plotly_mimetype"

In [ ]:
## User-defined parameters
d_start = 0
d_end = np.shape(m_xyz_pos)[0]
d_downsampling = 10
d_step = 200
d_max_ticks = 5

#d_step=int(d_step/d_downsampling)

# Apply slicing and downsampling
m_pos_sub = m_xyz_pos[d_start:d_end:d_downsampling].copy()
m_ypr_sub = m_ypr[d_start:d_end:d_downsampling].copy()
v_timestamp_sub = v_timestamp[d_start:d_end:d_downsampling].copy()

fig = f_create_auv_orientation_animation_plotly(m_pos_sub, m_ypr_sub, v_timestamp_sub,int(d_step/d_downsampling))
fig.show()

In [ ]:
# Ensure inline backend and no widgets are still "running"
plt.close('all')
%matplotlib inline

(VerticalAndHorizontalSonarDataVisualization)=
## Vertical and horizontal SONAR data visualization

This section presents different views of the sonar data:
- Static Polar view: received energy per angle
- Animated Polar view: received energy per range over time
- Sonar as an image of received energy per range over time
- Full view of sonar date with AUV trajectory path

<!-- The received signals can be seen over time as an image. Each waveform can be plotted as curves. The Y axis from top to bottom represents the distance from which the energy is received. The X axis is the time. -->

### Load the pre-processed sonar data

In [ ]:
# Load only the needed variables
glob = globals()
f_load_var (os.path.join('.','data','Pickle_dataset'), glob,
            ['m_beam_data_micron',       # (397, 45587) Received sonar data per range (397 bins from 0 to 20 meters) over time (associated with scanning angles) in dB
             'v_angles_rad_micron',      # associated scanning angle over time in radian
             'v_range_micron',           # Range scale from 0 to 20 over 397 bins
             'm_beam_data_micron_clean', # Cleaned sonar data
             'm_xyz_pos_micron',         # XYZ 3D position of the horizontal Micron sonar
             'v_timestamp_micron',
             'm_ypr_micron'])      # Timestamp associated with each sonar position
del glob

In [ ]:
# Starting when the sonar is 0º equivalent to the index 194 for the horizontal sonar
d_start_micron = 194

v_figsize = (8,8)

# Observing 1 period (200 points)
d_end_micron = d_start_micron + 200
m_sonar = m_beam_data_micron[:,d_start_micron:d_end_micron]
v_angle = v_angles_rad_micron[d_start_micron:d_end_micron]
f_plot_polar_sonar_micron(m_sonar, v_angle)

In [ ]:
d_image_width = 200
d_start_micron = 194
d_end_micron = d_start_micron + d_image_width
m_sonar_micron = m_beam_data_micron[:, d_start_micron:d_end_micron]
v_angles_rad = v_angles_rad_micron

v_figsize = (12,6)

f_plot_horizontal_sonar_with_angles(m_sonar_micron, v_angles_rad, d_start_micron, d_image_width, v_range_micron, v_figsize)

In [ ]:
# Load only the needed variables
glob = globals()
f_load_var (os.path.join('.','data','Pickle_dataset'), glob, 
            ['m_beam_data_seaking', 
             'v_angles_rad_seaking',
             'v_range_seaking',
             'm_xyz_pos_seaking',
             'v_timestamp_seaking',
             'm_beam_data_seaking_clean'])
del glob

In [ ]:
# Starting when the sonar is 0º equivalent to the index 95 for the vertical sonar
d_start_seaking = 95

# Observing 1 period (201 points)
d_end_seaking = d_start_seaking + 201

# Figure size
v_figsize = (8,8)

# Extracting the wanted part
m_sonar = m_beam_data_seaking[:,d_start_seaking:d_end_seaking]
v_angle = v_angles_rad_seaking[d_start_seaking:d_end_seaking]
f_plot_polar_sonar_seaking(m_sonar, v_angle, v_figsize)

In [ ]:
d_image_width = 202 # 1 period of 201 index + 1 point to display the full 360º

# Starting when the sonar is 0º equivalent to the index 95 for the vertical sonar
d_start_seaking = 95 
d_end_seaking = d_start_seaking + d_image_width

m_sonar_seaking = m_beam_data_seaking[:, d_start_seaking:d_end_seaking]
v_angles_rad = v_angles_rad_seaking
f_plot_vertical_sonar_with_angles(m_sonar_seaking, v_angles_rad, d_start_seaking, d_image_width, v_range_seaking)

### Observing several sonar revolution
> description...

### At the beginning of the survey

In [ ]:
d_image_width = 1000
d_start_micron = 1794
# d_end_micron = d_start_micron + d_image_width
m_sonar_micron = m_beam_data_micron[:, d_start_micron:d_start_micron+d_image_width]
v_figsize = (16,6)
f_plot_horizontal_sonar(m_sonar_micron, d_start_micron, d_image_width, v_range_micron, v_figsize)

In [ ]:
d_image_width = 800
d_start_micron = 194 # Angle is 0 rad at index 194 and period every 200

d_end_micron = d_start_micron + d_image_width

# Put the seaking start accordingly to the micron for consistency
d_start_seaking = np.argmin(np.abs(v_timestamp_seaking - v_timestamp_micron[d_start_micron]))
d_end_seaking = np.argmin(np.abs(v_timestamp_seaking - v_timestamp_micron[d_end_micron]))

d_image_width = d_end_seaking - d_start_seaking + 1

# d_start_seaking = 95
# d_end_seaking = d_start_seaking + d_image_width
m_sonar_seaking = m_beam_data_seaking[:, d_start_seaking:d_start_seaking+d_image_width]
v_figsize = (16,6)
f_plot_vertical_sonar(m_sonar_seaking, d_start_seaking, d_image_width, v_range_seaking, v_figsize)

In [ ]:
d_image_width = 1400
d_start_seaking = 95
# d_end_seaking = d_start_seaking + d_image_width
m_sonar_seaking = m_beam_data_seaking[:, d_start_seaking:d_start_seaking+d_image_width]
v_figsize = (16,6)
f_plot_vertical_sonar(m_sonar_seaking, d_start_seaking, d_image_width, v_range_seaking, v_figsize)

### In the middle of the survey

In [ ]:
d_image_width = 800
d_start_micron = 194 + 22400  # Angle is 0 rad at index 194 and period every 200
# d_end_micron = d_start_micron + d_image_width
m_sonar_micron = m_beam_data_micron[:, d_start_micron:d_start_micron+d_image_width]
v_figsize = (16,6)
f_plot_horizontal_sonar(m_sonar_micron, d_start_micron, d_image_width, v_range_micron, v_figsize)

In [ ]:
d_start_micron = 194 + 22400  # Angle is 0 rad at index 194 and period every 200
d_end_micron = d_start_micron + 1400
m_sonar_micron = m_beam_data_micron[:, d_start_micron:d_end_micron]
f_plot_horizontal_sonar(m_sonar_micron, d_start_micron, d_image_width, v_range_micron)

In [ ]:
d_image_width = 800
d_start_micron = 194 + 22400 # Angle is 0 rad at index 194 and period every 200

d_end_micron = d_start_micron + d_image_width

# Put the seaking start accordingly to the micron for consistency
d_start_seaking = np.argmin(np.abs(v_timestamp_seaking - v_timestamp_micron[d_start_micron]))
d_end_seaking = np.argmin(np.abs(v_timestamp_seaking - v_timestamp_micron[d_end_micron]))

d_image_width = d_end_seaking - d_start_seaking + 1

# d_start_seaking = 95
# d_end_seaking = d_start_seaking + d_image_width
m_sonar_seaking = m_beam_data_seaking[:, d_start_seaking:d_start_seaking+d_image_width]
v_figsize = (16,6)
f_plot_vertical_sonar(m_sonar_seaking, d_start_seaking, d_image_width, v_range_seaking, v_figsize)

## Data cleaning
> Work in progress...

In [ ]:
plt.style.use('default')
fig, ax = plt.subplots(1, 2, figsize=(16, 6))
ax[0].set_title("Horizontal Sonar back AUV body frame detection")
#ax[0].imshow(m_beam_data_micron[0:70, 594-35:594+35]) # 4159:4229])

# Determine image extent in terms of the date numbers on the x-axis
extent = [
    0,            # x_min
    70,           # x_max
    v_range_micron[-1],           # y_max
    v_range_micron[0]             # y_min (so 0m is at the top)
]

# Plot using imshow
h_im = ax[0].imshow(
    m_beam_data_micron[0:70, 594-35:594+35],
    aspect='auto',
    interpolation='none',
    extent=extent
)

# Measured values
d_length_back_start = -35
d_length_back_end = 35
d_length_back_beam = 70

d_start_micron = 22994

ax[1].set_title("Horizontal Sonar back AUV body frame detection")
ax[1].imshow(m_beam_data_micron[0:70, d_start_micron - 35 : d_start_micron + 35])

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(16, 9))
ax[0].set_title("Horizontal Sonar Front AUV Body Frame Detection")
ax[0].imshow(m_beam_data_micron[0:20, 4064:4133])

# Measured values
d_length_front_start = -35
d_length_front_end = 35
d_length_front_beam = 30

d_start_micron = 22994 + 100

ax[1].set_title("Horizontal Sonar Back AUV Body Frame Detection")
ax[1].imshow(m_beam_data_micron[0:20, d_start_micron - 35 : d_start_micron + 35])

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(16, 9))
ax[0].set_title("Horizontal Sonar back AUV body frame detection")
ax[0].imshow(
    m_beam_data_micron[
        0:d_length_back_beam, 1794 + d_length_back_start : 1794 + d_length_back_end #594 + d_length_back_start : 594 + d_length_back_end
    ]
)

#1794

ax[1].set_title("Horizontal Sonar front AUV body frame detection")
ax[1].imshow(
    m_beam_data_micron[
        0:d_length_front_beam, 1894 + d_length_front_start : 1894 + d_length_front_end #4099 + d_length_front_start : 4099 + d_length_front_end
    ]
)

### Interactive analysis
Widget to update the part of the trajectory in color associated with the sonar image below

In [ ]:
plt.close('all')
%matplotlib inline

In [ ]:
f_micron_with_trajectory(m_xyz_pos_micron, m_beam_data_micron, v_range_micron)

In [ ]:
f_seaking_with_trajectory(m_xyz_pos_seaking, m_beam_data_seaking, v_range_seaking)

In [ ]:
# Load only the needed variables
glob = globals()
f_load_var (os.path.join('.','data','Pickle_dataset'), glob, 
            ['m_beam_data_seaking', 
             'v_angles_rad_seaking',
             'v_range_seaking',
             'm_xyz_pos_seaking',
             'm_ypr_seaking',
             'v_timestamp_seaking',
             'm_beam_data_seaking_clean'])
del glob

## Animation scanning sonar 

In [ ]:
fig = plt.figure(figsize=(16,9))
ax = fig.add_subplot(projection="3d", proj_type="ortho")

# Put the axis in color and names them (X,Y,Z)
colors = ("#FF6666", "#005533", "#1199EE")  # Colorblind-safe RGB

for i, (axis, c) in enumerate(zip((ax.xaxis, ax.yaxis, ax.zaxis),
                                  colors)):
    axlabel = axis.axis_name
    axis.set_label_text(axlabel)
    axis.label.set_color(c)
    axis.line.set_color(c)
    axis.set_tick_params(colors=c)

d_start_seaking = 32456
d_end_seaking = d_start_seaking + 2000

# Plot trajectory with beginning and end
m_pos_seaking = m_xyz_pos_seaking[d_start_seaking:d_end_seaking,:].copy()
m_pos_seaking[:,2] = -m_pos_seaking[:,2]

trajectory = m_pos_seaking.copy()

# Setting the axes properties
d_margin = 15
ax.set_xlim3d([np.min(trajectory[:,0])-d_margin, np.max(trajectory[:,0])+d_margin])
ax.set_xlabel('X')

ax.set_ylim3d([np.min(trajectory[:,1])-d_margin, np.max(trajectory[:,1])+d_margin])
ax.set_ylabel('Y')

ax.set_zlim3d([np.min(trajectory[:,2])-d_margin, np.max(trajectory[:,2])+d_margin])
ax.set_zlabel('Z')

f_plot_3d_trajectory(ax,m_pos_seaking)
ax.set_title("(y,z) view vertical sonar scanning (Starting at 0°)")

m_aux_pc_seaking,l_intensity_seaking = f_pointCloud(
        m_xyz_pos_seaking[d_start_seaking:d_end_seaking,:], 
        m_ypr_seaking[d_start_seaking:d_end_seaking,:], 
        m_beam_data_seaking_clean[:,d_start_seaking:d_end_seaking], 
        v_angles_rad_seaking[d_start_seaking:d_end_seaking], 
        d_max_range=10, 
        d_thresh=0,b_return_intensity=True)

seaking_color = np.concatenate(l_intensity_seaking)

global p_pos
global graph
d_ind=0
p_pos = ax.scatter(m_pos_seaking[d_ind,0],m_pos_seaking[d_ind,1],m_pos_seaking[d_ind,2],color='blue')
#grap, = ax.plot(m_aux_pc_seaking[:,0], m_aux_pc_seaking[:,1], m_aux_pc_seaking[:,2], linestyle="",marker="o")

# ax.legend()
ax.view_init(20,65)

plt.close(fig)

graph = ax.scatter(m_aux_pc_seaking[:,0], m_aux_pc_seaking[:,1], m_aux_pc_seaking[:,2], s=1, c=seaking_color, cmap='jet')


l_index_seaking = [len(v_elem) for v_elem in l_intensity_seaking]
l_index_seaking = np.concatenate(([0],np.cumsum(l_index_seaking)))


#p = ax.scatter(m_aux_pc_seaking[:,0], m_aux_pc_seaking[:,1], m_aux_pc_seaking[:,2], s=1, c=seaking_color, cmap='jet')

def f_update_anim_sonar(d_ind):
    
    global graph
    global p_pos
    graph.remove()
    p_pos.remove()
    # Create the plot and get the graph object
    graph = ax.scatter(m_aux_pc_seaking[:l_index_seaking[d_ind],0], m_aux_pc_seaking[:l_index_seaking[d_ind],1], m_aux_pc_seaking[:l_index_seaking[d_ind],2], s=1, c=seaking_color[:l_index_seaking[d_ind]], cmap='jet')
    p_pos = ax.scatter(m_pos_seaking[d_ind,0],m_pos_seaking[d_ind,1],m_pos_seaking[d_ind,2],color='yellow')
    # Remove the previous plot and current plot
    # graph.set_visible(False)

    # Plot the new plot
    # graph = ax.scatter(m_aux_pc_seaking[:d_ind,0], m_aux_pc_seaking[:d_ind,1], m_aux_pc_seaking[:d_ind,2], s=1, c=seaking_color[:d_ind], cmap='jet')
    #graph._offsets3d = (m_aux_pc_seaking[d_ind:,0], m_aux_pc_seaking[d_ind:,1], m_aux_pc_seaking[d_ind:,2])
    
    #p = ax.scatter(m_aux_pc_seaking[:,0], m_aux_pc_seaking[:,1], m_aux_pc_seaking[:,2], s=1, c=seaking_color, cmap='jet')
    fig.canvas.flush_events()
    fig.canvas.draw_idle()
    fig.canvas.draw()
    display(fig)

In [ ]:
widget_seaking_3D = interactive(
    f_update_anim_sonar,
    d_ind=widgets.IntSlider(min=1,max=d_end_seaking-d_start_seaking-1,step=1,value=10)) # widgets.FloatSlider(min=0,max=1,step=0.05,value=0.9))

In [ ]:
display(widget_seaking_3D)

## Cleaning the SONAR data

#### Mask creation from intensity
When the wall detection is close by, removing the body frame detection with a simple rectangle could also remove the detected wall.
The idea is to create a mask of the body frame detection with intensity and subtract this intensity from the total data.
In the case of font body detection, the wall detection doesn't seem to overlap the AUV body frame detection so a simple replacement of values should suffice.  

Let's find a spot where the detected walls are very far to isolate only the body frame detection

In [ ]:
m_mask_instensity_back = m_beam_data_micron[0:70, 594 - 35 : 594 + 35].copy()
d_length_back_beam = 70
d_length_back_start = -35
d_length_back_end = 35

m_mask_intensity_front = m_beam_data_micron[0:30, 4064:4134].copy()
d_length_front_start = -35
d_length_front_end = 35
d_length_front_beam = 30

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(16, 9))
ax[0].set_title("Horizontal Sonar back AUV body frame detection")
ax[0].imshow(m_beam_data_micron[0:d_length_back_beam, 594 + d_length_back_start : 594 + d_length_back_end])

ax[1].set_title("Horizontal Sonar front AUV body frame detection")
ax[1].imshow(m_beam_data_micron[0:d_length_front_beam, 4099 + d_length_front_start : 4099 + d_length_front_end]);

In [ ]:
%matplotlib inline
plt.close('all')

d_start_micron = 22994
d_end_micron = d_start_micron + 1000 + 1 # + 1 that display the '1000'

fig, ax = plt.subplots(2, 1, figsize=(16, 9))
ax[0].set_title("Before cleaning")
ax[0].imshow(m_beam_data_micron[:, d_start_micron:d_end_micron])

ax[1].set_title("After cleaning")
ax[1].imshow(m_beam_data_micron_clean[:, d_start_micron:d_end_micron]);

## 3D point cloud from combining both SONAR data with a uniform colormap based on cleaned data

In [ ]:
# Load only the needed variables
glob = globals()
f_load_var (os.path.join('.','data','Pickle_dataset'), glob, ['v_timestamp_seaking', 'v_timestamp_micron','m_ypr_micron','m_ypr_seaking'])
del glob

In [ ]:
d_start_micron = 0 
d_end_micron =  d_start_micron + np.shape(m_beam_data_micron_clean)[1]-1 #6500

# Put the seaking start accordingly to the micron one
d_start_seaking = np.argmin(np.abs(v_timestamp_seaking - v_timestamp_micron[d_start_micron]))
d_end_seaking = np.argmin(np.abs(v_timestamp_seaking - v_timestamp_micron[d_end_micron]))

# The max values are selected when d_tresh = -1
m_point_cloud_micron_max = f_pointCloud(
    m_xyz_pos_micron[d_start_micron:d_end_micron,:], 
    m_ypr_micron[d_start_micron:d_end_micron,:], 
    m_beam_data_micron_clean[:,d_start_micron:d_end_micron], 
    v_angles_rad_micron[d_start_micron:d_end_micron], 
    d_max_range=20, 
    d_thresh=-1)

m_point_cloud_seaking_max = f_pointCloud(
    m_xyz_pos_seaking[d_start_seaking:d_end_seaking,:], 
    m_ypr_seaking[d_start_seaking:d_end_seaking,:], 
    m_beam_data_seaking_clean[:,d_start_seaking:d_end_seaking], 
    v_angles_rad_seaking[d_start_seaking:d_end_seaking], 
    d_max_range=10, 
    d_thresh=-1)

In [ ]:
%matplotlib inline
plt.close('all')
fig = plt.figure(figsize=(25,12))
ax = fig.add_subplot(projection="3d", proj_type="ortho",box_aspect=(5,4,2))

# Put the axis in color and name them (X,Y,Z)
colors = ("#FF6666", "#005533", "#1199EE")  # Colorblind-safe RGB
for i, (axis, c) in enumerate(zip((ax.xaxis, ax.yaxis, ax.zaxis),
                                  colors)):
    axlabel = axis.axis_name
    axis.set_label_text(axlabel)
    axis.label.set_color(c)
    axis.line.set_color(c)
    axis.set_tick_params(colors=c)

m_pos_micron = m_xyz_pos_micron[d_start_micron:d_end_micron,:].copy()
m_pos_micron[:,2] = -m_pos_micron[:,2]

m_pos_seaking = m_xyz_pos_seaking[d_start_seaking:d_end_seaking,:].copy()
m_pos_seaking[:,2] = -m_pos_seaking[:,2]

micron_color = m_point_cloud_micron_max[:,2 ] #np.concatenate(l_intensity_micron)
seaking_color = m_point_cloud_seaking_max[:,2]

# Put the same max and min value to have the same color map
d_min = np.min((np.min(micron_color),np.min(seaking_color)))
d_max = np.max((np.max(micron_color),np.max(seaking_color)))
micron_color[micron_color == np.min(micron_color)] = d_min
seaking_color[seaking_color == np.min(seaking_color)] = d_min

micron_color[micron_color == np.max(micron_color)] = d_max
seaking_color[seaking_color == np.max(seaking_color)] = d_max

# Plot trajectory with beginning and end
# f_plot_3d_trajectory(ax,m_pos_micron)
f_plot_3d_trajectory(ax,m_pos_seaking)

p = ax.scatter(m_point_cloud_micron_max[:,0], m_point_cloud_micron_max[:,1], m_point_cloud_micron_max[:,2], s=1, c=micron_color, cmap='jet', label='Micron')
p = ax.scatter(m_point_cloud_seaking_max[:,0], m_point_cloud_seaking_max[:,1], m_point_cloud_seaking_max[:,2], s=1, c=seaking_color, cmap='jet', label='Seaking')

ax.set_title("Perspective view map ")
# ax.legend()
ax.view_init(30,45)
ax.invert_yaxis()

ax.set_title("3D Point cloud from Horizontal and vertical SONAR", fontsize=30)
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')

ax.dist = 0
fig.colorbar(p,shrink=0.6)
plt.show()

::::{note} Link to external notebooks
(01_Preprocessing)=
:::{card} 01_Preprocessing.ipynb
:link: ./01_Preprocessing.ipynb
See all the processes that have been done on the data from the CSV dataset to the pickle file
:::

(ROS2csv)=
:::{card} ROSbag2csv.ipynb
:link: ./ROSbag2csv.ipynb
See how to extract ROS bag data and convert each topic to a CSV file.
:::
::::